In [1]:
import numpy as np
a = np.array([(3, 6), (3, 7), (9, 9)])
b = np.array([(3, 6), (4, 6), (6, 6)])
w = 10
def coords_to_indices(coords, width):
    return coords[:, 1] * width + coords[:, 0]
def indices_to_coords(indices, width):
    y = indices // width
    x = indices % width
    return np.vstack((x, y)).T

In [2]:
import ctypes
import numpy as np

def call_cuda_find_intersection(list1, list2, deltax, deltay, width):
    lib = ctypes.CDLL('./findIntersection.so')

    list1_indices, list2_indices = coords_to_indices(list1, width).astype(np.int32), coords_to_indices(list2, width).astype(np.int32)

    result = np.zeros(len(list1_indices), dtype=np.int32)
    
    lib.launchFindIntersection.argtypes = [
        np.ctypeslib.ndpointer(dtype=np.int32),  # list1
        np.ctypeslib.ndpointer(dtype=np.int32),  # list2
        ctypes.c_int,  # list1Size
        ctypes.c_int,  # list2Size
        ctypes.c_int,  # deltax
        ctypes.c_int,  # deltay
        np.ctypeslib.ndpointer(dtype=np.int32)   # result
    ]

    lib.launchFindIntersection(
        list1_indices, 
        list2_indices,
        len(list1_indices), 
        len(list2_indices),
        deltax, 
        deltay,
        result
    )

    print(result)

    intersecting_points = [(list1_indices[i * 2] + deltax, list1_indices[i * 2 + 1] + deltay) for i in range(len(result)) if result[i] == 1]
    return intersecting_points

In [3]:
call_cuda_find_intersection(a, b, 0, 0, w)

[63, 73, 99]
[63, 64, 66]
[63 64 66]


[]